# Setup

Import neccessary libraries

In [4]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.16.1
Eager mode:  True
Hub version:  0.16.1
GPU is NOT AVAILABLE


# Load Dataset

The data is self-labelled data extracted from Scopus database. It is available on the C-drive

In [12]:
# Load and read datasets

papers = pd.read_excel('C:/Users/Saleeh/Documents/Salford University/Lectures/AI - Natural Language Processing/Assessment/Dataset/Scopus Extract/Scopus dataset3.xlsx')

In [13]:
# View first 5 rows of data 

papers.head()

,Authors,Title,Year,Source title,Link,Abstract,Author Keywords,Subject
0,Belguith S.; Kaaniche N.; Laurent M.; Jemai A....,Accountable privacy preserving attribute based...,2020,Journal of Parallel and Distributed Computing,https://www.scopus.com/inward/record.uri?eid=2...,"In this paper, we propose an accountable priva...",Accountability; Attribute based encryption; At...,Mathematics
1,Kumaran G.; Sivaraj R.; Ramachandra Prasad V.;...,Finite difference computation of free magneto-...,2021,Physica Scripta,https://www.scopus.com/inward/record.uri?eid=2...,It is essential to account the variability in ...,Chemical reaction; Keller box method; Powell-e...,Mathematics
2,Zahedi E.; Saraee M.; Masoumi F.S.; Yazdinejad M.,Regularized Contrastive Masked Autoencoder Mod...,2023,Algorithms,https://www.scopus.com/inward/record.uri?eid=2...,"Unsupervised anomalous sound detection, especi...",anomalous sound detection; Contrastive learnin...,Mathematics
3,Oladipupo M.A.; Obuzor P.C.; Bamgbade B.J.; Ol...,An Automated Python Script for Data Cleaning a...,2023,Informatica (Slovenia),https://www.scopus.com/inward/record.uri?eid=2...,Every employee in the company who deals with d...,automation; data cleaning; financial dataset; ...,Mathematics
4,Roy A.K.; Debnath S.; Bég O.A.,Transient solute dispersion in electro-osmotic...,2023,ZAMM Zeitschrift fur Angewandte Mathematik und...,https://www.scopus.com/inward/record.uri?eid=2...,The transport of a neutral solute in incompres...,NaN,Mathematics


In [14]:
#papers.describe(include = 'all')

In [15]:
# Remove columns that does not relate to the task

papers = papers.drop(columns=['Authors','Source title','Link', 'Author Keywords', 'Title', 'Year'], axis=1)

#print out the first five rows of papers
papers.head()

,Abstract,Subject
0,"In this paper, we propose an accountable priva...",Mathematics
1,It is essential to account the variability in ...,Mathematics
2,"Unsupervised anomalous sound detection, especi...",Mathematics
3,Every employee in the company who deals with d...,Mathematics
4,The transport of a neutral solute in incompres...,Mathematics


In [17]:
#Assign Label

def assign_label(subject):
    if subject == "Arts and Humanity":
        return 0
    elif subject == "Business management":
        return 1
    elif subject == "Computer science and engineering":
        return 2
    elif subject == "Mathematics":
        return 3
    elif subject == "Medicine":
        return 4
    else:
        return None  # Handle other cases if needed

# Apply the function to your DataFrame
papers["label"] = papers["Subject"].apply(assign_label)



papers.tail()

,Abstract,Subject,label
1137,Historically the British Trades Union Congress...,Arts and Humanity,0
1138,‘Emotional Weather Report’ is a song by Tom Wa...,Arts and Humanity,0
1139,Even though the rise of the neural machine tra...,Arts and Humanity,0
1140,This article responds to McCloud’s theoretical...,Arts and Humanity,0
1141,Background and objectives: Visual Schema Displ...,Arts and Humanity,0


In [18]:
# Drop Subject column

papers = papers.drop(columns=['Subject'], axis=1)

#print out the first five rows of papers
papers.head()

,Abstract,label
0,"In this paper, we propose an accountable priva...",3
1,It is essential to account the variability in ...,3
2,"Unsupervised anomalous sound detection, especi...",3
3,Every employee in the company who deals with d...,3
4,The transport of a neutral solute in incompres...,3


In [20]:
# Check if there is any missing data

papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Abstract  1142 non-null   object
 1   label     1142 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 18.0+ KB


# Prepare data for model training

In [24]:
x = papers.Abstract
y = papers.label

print(x)

0       In this paper, we propose an accountable priva...
1       It is essential to account the variability in ...
2       Unsupervised anomalous sound detection, especi...
3       Every employee in the company who deals with d...
4       The transport of a neutral solute in incompres...
                              ...                        
1137    Historically the British Trades Union Congress...
1138    ‘Emotional Weather Report’ is a song by Tom Wa...
1139    Even though the rise of the neural machine tra...
1140    This article responds to McCloud’s theoretical...
1141    Background and objectives: Visual Schema Displ...
Name: Abstract, Length: 1142, dtype: object


In [26]:
#Data splitting
# 20% of the data will be reserved for testing

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(papers,y,test_size=0.2,random_state=42)

In [33]:
x_train.shape,y_test.shape

((913, 2), (229,))

In [29]:
# Print first 10 examples for training set

print(x_train[:10])

                                               Abstract  label
12    This letter presents the design and developmen...      3
758   Introduction: Bereavement is often difficult f...      4
636   Background: Paediatric mortality rates in the ...      4
1108  2019 sees the 50th anniversary of the iconic B...      0
743   This study makes use of a cohesive yet innovat...      4
620   Purpose: This is the first study that aimed to...      4
673   The International Virus Bioinformatics Meeting...      4
993   Wildfires in Siberia are documented to have in...      0
796   The countermovement jump (CMJ) is regularly te...      4
184   This study aims to provide a method that will ...      2


# Model biulding

In [ ]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_example[])